In [29]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_csv("data/bpost.csv").dropna().drop('Forcast', axis=1)
df

,Datum,Actual
0,03/08/2020,466.0
1,04/08/2020,633.0
2,05/08/2020,685.0
3,06/08/2020,601.0
4,07/08/2020,607.0
...,...,...
124,25/01/2021,856.0
125,26/01/2021,853.0
126,27/01/2021,918.0
127,28/01/2021,803.0


In [14]:
df.Datum.nunique()

129

In [34]:
from avatar import WranglingLanguage, Expander
from avatar.transformations import TimeFeatures
from avatar.settings import Settings
from avatar.filter import UniqueFilter

Settings.verbose = True

In [35]:
language = WranglingLanguage([TimeFeatures])
expander = Expander(language)
expanded = UniqueFilter().select(expander.expand(df, exclude=["Actual"]))
expanded = expanded.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64'])
expanded.head()

100%|██████████| 2/2 [00:01<00:00,  1.14it/s]


,Actual,TimeFeatures()(Datum)_w,TimeFeatures()(Datum)_-d,TimeFeatures()(Datum)_-m,TimeFeatures()(Datum)_-y,TimeFeatures()(Datum)_Y,TimeFeatures()(Datum)_-j,TimeFeatures()(Datum)_U,TimeFeatures()(Datum)_W
0,466.0,0,8,3,20,2020,68,10,9
1,633.0,3,8,4,20,2020,99,14,14
2,685.0,5,8,5,20,2020,129,18,18
3,601.0,1,8,6,20,2020,160,23,23
4,607.0,3,8,7,20,2020,190,27,27


In [49]:
from avatar.supervised import AnnealingTournament, LegacyTournament, DefaultJudge

In [37]:
legacy = LegacyTournament(games=400)
legacy.initialise(expanded, target="Actual")
legacy.play()

100%|██████████| 400/400 [00:08<00:00, 45.71it/s]


In [38]:
legacy.results

{'TimeFeatures()(Datum)_Y': 0.05750133809330579,
 'TimeFeatures()(Datum)_-d': 0.2357857294334564,
 'TimeFeatures()(Datum)_U': 0.13489988915144133,
 'TimeFeatures()(Datum)_w': 0.057683393822469835,
 'TimeFeatures()(Datum)_-m': 0.07145651335037817,
 'TimeFeatures()(Datum)_W': 0.13029075065731285,
 'TimeFeatures()(Datum)_-j': 0.2749407581146985,
 'TimeFeatures()(Datum)_-y': 0.03744162737693718}

In [51]:
from avatar.supervised import Game, AveragePool

anneal = AnnealingTournament(
    Game(rounds=1, judge=DefaultJudge()),
    AveragePool(),
    games=100,
    exploration=0.5,
    teamsize=4
)
anneal.initialise(expanded, target="Actual")
anneal.play()



  0%|          | 0/100 [00:00<?, ?it/s]

8:   1%|          | 1/100 [00:00<00:00, 1234.71it/s]

7:   2%|▏         | 2/100 [00:00<00:01, 57.96it/s]  

7:   3%|▎         | 3/100 [00:00<00:02, 45.37it/s]

7:   4%|▍         | 4/100 [00:00<00:02, 37.57it/s]

7:   4%|▍         | 4/100 [00:00<00:02, 37.57it/s]

7:   5%|▌         | 5/100 [00:00<00:02, 37.57it/s]

7:   6%|▌         | 6/100 [00:00<00:02, 37.57it/s]

7:   7%|▋         | 7/100 [00:00<00:02, 37.57it/s]

7:   8%|▊         | 8/100 [00:00<00:04, 22.92it/s]

7:   8%|▊         | 8/100 [00:00<00:04, 22.92it/s]

7:   9%|▉         | 9/100 [00:00<00:03, 22.92it/s]

7:  10%|█         | 10/100 [00:00<00:03, 22.92it/s]

7:  11%|█         | 11/100 [00:00<00:05, 17.68it/s]

7:  11%|█         | 11/100 [00:00<00:05, 17.68it/s]

7:  12%|█▏        | 12/100 [00:00<00:04, 17.68it/s]

7:  13%|█▎        | 13/100 [00:00<00:04, 17.68it/s]

7:  14%|█▍        | 14/100 [00:00<00:05, 15.69it/s]

7:  14%|█▍        | 14/100 [00:00<00:05, 15.69it/s]

7:  15%|█▌

4:  94%|█████████▍| 94/100 [00:08<00:00, 10.59it/s]

4:  95%|█████████▌| 95/100 [00:08<00:00, 10.59it/s]

4:  96%|█████████▌| 96/100 [00:08<00:00, 11.89it/s]

4:  96%|█████████▌| 96/100 [00:08<00:00, 11.89it/s]

4:  97%|█████████▋| 97/100 [00:08<00:00, 11.89it/s]

4:  98%|█████████▊| 98/100 [00:08<00:00, 11.73it/s]

4:  98%|█████████▊| 98/100 [00:08<00:00, 11.73it/s]

4:  99%|█████████▉| 99/100 [00:08<00:00, 11.73it/s]

4: 100%|██████████| 100/100 [00:08<00:00, 12.27it/s]

4: 100%|██████████| 100/100 [00:08<00:00, 12.27it/s]

In [52]:
anneal.results

{'TimeFeatures()(Datum)_Y': 0.00893538931796997,
 'TimeFeatures()(Datum)_-d': 0.2888855666455491,
 'TimeFeatures()(Datum)_U': 0.05457953149181349,
 'TimeFeatures()(Datum)_w': 0.05559503777983226,
 'TimeFeatures()(Datum)_-m': 0.03353087099864055,
 'TimeFeatures()(Datum)_W': 0.0830821192789483,
 'TimeFeatures()(Datum)_-j': 0.46599895763488275,
 'TimeFeatures()(Datum)_-y': 0.009392526852363534}



4: 100%|██████████| 100/100 [00:27<00:00, 12.27it/s]

In [41]:
from avatar.evaluate import RankingEvaluator

In [42]:
estimator = RandomForestRegressor(max_depth=4)
evaluator = RankingEvaluator(estimator, max_features=10)

In [43]:
evaluator.fit(expanded, target="Actual", ranking=legacy.results)
evaluator.scores


4: 100%|██████████| 400/400 [00:46<00:00,  8.52it/s]

100%|██████████| 9/9 [00:12<00:00,  1.38s/it]


{1: {'scores': [-0.07068439655836153,
   -0.14504503081765457,
   -0.3003476368658234,
   0.019665460550896397,
   -0.39522748197645885],
  'features': ['TimeFeatures()(Datum)_-j']},
 2: {'scores': [0.3366916436594877,
   0.4425045182502322,
   0.14841320268516728,
   0.18318044375791936,
   0.25266156528246514],
  'features': ['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d']},
 3: {'scores': [0.28919582306445274,
   0.16908238496290043,
   0.10251187220457085,
   0.1789785948229407,
   0.08158350049205851],
  'features': ['TimeFeatures()(Datum)_-j',
   'TimeFeatures()(Datum)_-d',
   'TimeFeatures()(Datum)_U']},
 4: {'scores': [-0.023319055879807225,
   0.14606985643169673,
   0.26709867778813123,
   0.12216947503454459,
   0.011614565524274357],
  'features': ['TimeFeatures()(Datum)_-j',
   'TimeFeatures()(Datum)_-d',
   'TimeFeatures()(Datum)_U',
   'TimeFeatures()(Datum)_W']},
 5: {'scores': [0.39117891786537634,
   0.12120397452781424,
   0.4452189871372817,
   -0.0073463320

In [44]:
for k, v in _.items():
    print(k, np.mean(v['scores']))
    print(v['features'])
    print()

1 -0.1783278171334804
['TimeFeatures()(Datum)_-j']

2 0.2726902747270543
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d']

3 0.16427043510938463
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_U']

4 0.10472670377976792
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_U', 'TimeFeatures()(Datum)_W']

5 0.22225443009821166
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_U', 'TimeFeatures()(Datum)_W', 'TimeFeatures()(Datum)_-m']

6 0.20308969289563694
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_U', 'TimeFeatures()(Datum)_W', 'TimeFeatures()(Datum)_-m', 'TimeFeatures()(Datum)_w']

7 0.3479778748273305
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_U', 'TimeFeatures()(Datum)_W', 'TimeFeatures()(Datum)_-m', 'TimeFeatures()(Datum)_w', 'TimeFeatures()(Datum)_Y']

8 0.08732406005899032
['TimeFeatures()(Datum)_-j', 'TimeFeatur

In [53]:
evaluator.fit(expanded, target="Actual", ranking=anneal.results)
evaluator.scores




  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:01<00:13,  1.70s/it]


 22%|██▏       | 2/9 [00:03<00:13,  1.99s/it]


 33%|███▎      | 3/9 [00:05<00:10,  1.68s/it]


 44%|████▍     | 4/9 [00:06<00:07,  1.53s/it]


 56%|█████▌    | 5/9 [00:07<00:05,  1.42s/it]


 67%|██████▋   | 6/9 [00:08<00:03,  1.21s/it]


4: 100%|██████████| 100/100 [00:40<00:00,  2.47it/s][A



 89%|████████▉ | 8/9 [00:11<00:01,  1.30s/it]


100%|██████████| 9/9 [00:12<00:00,  1.42s/it]


  0%|          | 0/9 [00:00<?, ?it/s]

 11%|█         | 1/9 [00:02<00:22,  2.79s/it]

 22%|██▏       | 2/9 [00:05<00:17,  2.51s/it]

 33%|███▎      | 3/9 [00:06<00:12,  2.06s/it]

 44%|████▍     | 4/9 [00:08<00:09,  1.88s/it]

 56%|█████▌    | 5/9 [00:09<00:06,  1.72s/it]

 67%|██████▋   | 6/9 [00:11<00:05,  1.78s/it]

 78%|███████▊  | 7/9 [00:13<00:03,  1.80s/it]

 89%|████████▉ | 8/9 [00:15<00:01,  1.80s/it]

100%|██████████| 9/9 [00:16<00:00,  1.89s/it]


  0%|          | 0/9 [00:00<?, ?it/s]

 11%|█       

{1: {'scores': [0.22077669440672237,
   -0.32709566760854025,
   -0.21285394701507793,
   0.06747307535807841,
   -0.4798620129087487],
  'features': ['TimeFeatures()(Datum)_-j']},
 2: {'scores': [0.27668906672519256,
   -0.3734829524029105,
   -0.0020113331013809966,
   0.18595743406011772,
   0.2281691103313642],
  'features': ['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d']},
 3: {'scores': [0.1337571067608381,
   0.4038717872102926,
   0.3047926390513819,
   0.1693003444772606,
   0.12673144771537606],
  'features': ['TimeFeatures()(Datum)_-j',
   'TimeFeatures()(Datum)_-d',
   'TimeFeatures()(Datum)_W']},
 4: {'scores': [0.36120598178557206,
   0.15675192068691768,
   0.1898789236977027,
   0.2630643446352013,
   0.33341166526057203],
  'features': ['TimeFeatures()(Datum)_-j',
   'TimeFeatures()(Datum)_-d',
   'TimeFeatures()(Datum)_W',
   'TimeFeatures()(Datum)_w']},
 5: {'scores': [0.23328930700829797,
   0.2847659738052875,
   0.02265713027193641,
   0.36477960811092475

In [54]:
for k, v in _.items():
    print(k, np.mean(v['scores']))
    print(v['features'])
    print()

1 -0.14631237155351323
['TimeFeatures()(Datum)_-j']

2 0.06306426512247659
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d']

3 0.22769066504302984
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_W']

4 0.2608625672131931
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_W', 'TimeFeatures()(Datum)_w']

5 0.13663123330865531
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_W', 'TimeFeatures()(Datum)_w', 'TimeFeatures()(Datum)_U']

6 0.2543729277270289
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_W', 'TimeFeatures()(Datum)_w', 'TimeFeatures()(Datum)_U', 'TimeFeatures()(Datum)_-m']

7 0.0464533955761158
['TimeFeatures()(Datum)_-j', 'TimeFeatures()(Datum)_-d', 'TimeFeatures()(Datum)_W', 'TimeFeatures()(Datum)_w', 'TimeFeatures()(Datum)_U', 'TimeFeatures()(Datum)_-m', 'TimeFeatures()(Datum)_-y']

8 0.30722197009111324
['TimeFeatures()(Datum)_-j', 'TimeFeatur